#### Anthony Wilson
##### Week 6 Problem 1
##### 7/18/2020

The goal of this part is to write a script to compute the decision tree for an input dataset. You can assume that all attributes are numeric, except for the last attribute which is the class.

You should use the Information Gain based on Entropy for computing the best split value for each attribute. For the stopping criteria at a node, stop if the purity is at least 95% or stop if the node size is five or lower.

Note that the best way to implement the splits for numeric attributes is to sort the values of that attribute from smallest to largest. Then you can use the mid-point between two distinct (consecutive) values as the split test of the form A≤v. You can then update the class frequencies on both sides of the split and compute the split entropy for each decision. After comparing with the entropy for the node, you can find the best split for the current attribute. Now repeat the whole process for each numeric attribute at that node, and choose the best split over all attributes. Finally, split the data according to the best split, and repeat the whole method recursively, until the stopping conditions are met.

The decision tree should be printed in the following format:

Decision: Car <= 1.5 , Gain= 0.4591479

Decision: Age <= 22.5 , Gain= 0.9182958

Leaf: label= H purity= 1 size= 1

Leaf: label= L purity= 1 size= 2

Leaf: label= H purity= 1 size= 3

Note that each internal node, print the decision followed by the Information Gain, and for each leaf, print the majority label, purity of the leaf, and the size. The indentation indicates the tree level. All nodes at the same level of indentation (tabs) are at the same level in the tree. For the tree above, Car<=1.5 is the root decision. Its left child is Age<=22.5, and its right child is a leaf. Also, for Age≤22.5 its left and right children appear immediately below it.

You may test your program on the iris.txt dataset.

In [1]:
import numpy as np
import random as rd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

def readInTextFile(filename):
    """
    Reading in file and returning the results
    """
    _list_ = []
    i = 0
    # using 'with open' function, so that when finished looping file it closes
    with open(filename, 'r') as file:
        #loop through line by line to read file
        for line in file:
            # strip all extra unnecesary text like drop line "\n" 
            # use split to break up the values into a list
            s = line.strip().split( ",")
            _list_.append(s)
    return _list_
# bring in file data as a numpy list so that it is easier to slice 
data = pd.DataFrame(readInTextFile("iris.txt"), columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'label'])
# convert strings to floats 
for col in data.columns[:-1]:
    data[col] = data[col].astype(float)
# drop quoutes in label columns
data['label'] = data['label'].str.replace('"', '')
data.head()

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.9,3.0,4.2,1.5,Iris-versicolor
1,6.9,3.1,4.9,1.5,Iris-versicolor
2,6.6,2.9,4.6,1.3,Iris-versicolor
3,4.6,3.2,1.4,0.2,Iris-setosa
4,6.0,2.2,4.0,1.0,Iris-versicolor


In [2]:
def train_test_split(df, test_size):
    """
    train_test_split:
    Takes a pandas dataframe and splits it up into a training set and a test set, 
    based on the test size. It will then return the two dataframes.
    """
    # when test_size represents a probability between 0 and 1 resets test_size 
    # by the proportion sent to the function
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))
    # get a list of all the indexes    
    indices = df.index.to_list()
    # get a sample from the dataframe for the test data set
    test_indices= rd.sample(population = indices, k = test_size)
    # pull indices from dataframe and create the test dataframe
    test_df = df.loc[test_indices]
    # drop test dataframe indices for the training dataset
    train_df = df.drop(test_indices)
    return train_df, test_df

In [3]:
def calc_purity(data):
    """
    calc_purity:
    Checks the purity of the data, which is a numpy array and returns if it is pure or not. 
    If the max probabilty of the number of labels from each class returns higher then the 
    given threshold return true. 
    """
    labels = data[:,-1]
    counts = np.unique(labels, return_counts=True)[1]

    probabilities = counts/counts.sum()
    return max(probabilities)


In [4]:
def classify_data(data):
    """
    classify_data:
    Takes numpy array data and returns label with the most counts in the data set. 
    """
    # Gather labels
    labels = data[:,-1]
    # Get each unique class and their counts
    uniq_classes, uniq_class_counts = np.unique(labels, return_counts=True)
    # Get the index with the most labels from uniq_class_counts
    index = uniq_class_counts.argmax()
    # Get label name
    classification = uniq_classes[index]
    return classification

In [5]:
def get_potential_splits(data):
    """
    get_potential_splits:
    For each column in the numpy array, besides the last one which is the label, 
    return a dictionary of all possibible splits. The potential_split is calculated over 
    a loop where the previous value within the column is added to the current and divided by 
    two. This is done for each column over earch row. 
    """
    # initiate potential_splits
    potential_splits= {}
    # Get the number of columns from the data set
    n_columns = data.shape[1]
    
    # For each column, besides the label create list of unique values
    for column_index in range(n_columns-1):
        potential_splits[column_index] = []
        values = data[:,column_index]
        unique_values = np.unique(values)
        # skip the first index of each column you can't split the first and the last
        for index in range(1,len(unique_values)):
            current_value = unique_values[index]
            previous_value = unique_values[index-1]
            potential_split = (current_value+previous_value)/2
            potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [6]:
def split_data(data, split_column, split_value):
    """
    split_data:
    Takes the numpy array data and splits based on the column index
    and a value pased through the function. It will create two new 
    numpy arrays based on the split value sent. There will two arrays 
    returned one arrayof data below the split value and one above split 
    value.
    """    
    split_col_values = data[:,split_column]
    data_below = data[split_col_values <= split_value]
    data_above = data[split_col_values > split_value]
    return data_below, data_above

In [7]:
def calc_entropy(data):
    """
    calc_entropy:
    Takes numpy array data and calculates the entropy. First it pulls the
    labels and counts, then it calculates the probabilities for each, 
    then it calculates the entropy by summing the probability times the 
    negative log of the probabilities and returns the value.
    """
    
    labels = data[:,-1]
    counts = np.unique(labels, return_counts=True)[1]

    probabilities = counts/counts.sum()
    impurity = sum(probabilities * -np.log2(probabilities))
    return impurity

In [8]:
def calc_overall_entropy(data_below, data_above):
    """
    calc_overall_entropy:
    Calculates the overall entropy for the data given the data below a certain value
    within a column and above that value. First we get the total length of each numpy 
    array passed for data above and below, then we calulate the probability for both
    and take the probability times the entropy of each and some them up to get the 
    overall entropy and return the value.
    """
    n_data = len(data_below) + len(data_above)

    p_data_below = len(data_below) / n_data
    p_data_above = len(data_above) / n_data
    overall_entropy =( p_data_below * calc_entropy(data_below)
                     +  p_data_above * calc_entropy(data_above))
    
    return overall_entropy

In [9]:
def determine_best_split(data,potential_splits):
    """
    determine_best_split:
    This goes through the numpy array data with all potential splits from the data
    based on column and each unique value within, and returns the best possible split
    by calculating the overall entropy if the data was to be split by the column and split value. 
    The function loops over each column and if the overall entropy is the smallest, the column and 
    split value is stored and returned. 
    """
    # information gain set to 0 to initialize
    best_gain = 0
    # Get current uncertainty for data to measure information gain
    current_uncertainty = calc_entropy(data)
    
    # loop through each column and value to determine best split
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            # Split the data based on the column and value
            data_below, data_above = split_data(data,column_index, value)
            # Get the current iterations uncertainty
            iteration_uncertainty = calc_overall_entropy(data_below, data_above)
            # Calculate the new information gain 
            gain = current_uncertainty - iteration_uncertainty
            # If the current iteration increases the information gain update best values
            if gain >= best_gain:
                best_gain = gain
                best_split_column = column_index
                best_split_value = value
    return best_split_column, best_split_value, best_gain

In [10]:
def decision_tree_algorithm(df, counter= 0, min_node = 5, threshold_purity = .95):
    """
    decision_tree_algorithm:
    This is a recursive function that takes a panda dataframe and determines the best way to split the 
    data. It first converts the dataframe into a numpy array, checks to see if it is pure if it is not
    then it will start to find the best split among all columns. It calls itself instead of using 
    a while loop. It will continue to recursively call its self until each node is pure. 
    """    
    # First step, if the counter=1 then setup the data set otherwise the data is already setup 
    # as a numpy array.
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df

    purity = calc_purity(data)
    node_size = len(data)
    # Check to see if the data is pure, if it is stop and return the classification to create a leaf.
    if (purity >= threshold_purity) or (node_size < min_node):
        classification = classify_data(data)
        leaf = 'Leaf: label = {} purity = {} size = {}'.format(classification, purity, node_size)
        return leaf
    # recursive part
    else:
        counter += 1
        
        # helper functions
        potential_splits = get_potential_splits(data)
        split_column, split_value, information_gain = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        question = "Decision: {} <= {} , Gain = {}".format( feature_name, split_value, information_gain)
        sub_tree = {question: []}
        
        # find answers (recursion)
        left_node = decision_tree_algorithm(data_below, counter, min_node, threshold_purity)
        right_node = decision_tree_algorithm(data_above, counter, min_node,threshold_purity)
        
        if left_node == right_node:
            sub_tree = left_node
        else: 
            sub_tree[question].append(left_node)
            sub_tree[question].append(right_node)
        
        return sub_tree

In [11]:
def classify_example(example,tree):
    """
    classify_example:
    Takes a value from the data as list with column values and recursively goes through the decision tree
    to classify each row. 
    """
    # Pull inequality with columns
    q = list(tree.keys())[0]
    # Get the subtree index by subtracting 1 from True or False to get the inverse
    sub_tree_index = 1-int(example[q.split(' ')[1]] <= float(q.split(' ')[3]))
    # Get the index
    response = list(tree.values())[0][sub_tree_index]
    # If the response is not a dictionary return the repsonse
    if type(response) != dict:
        return response.split(' ')[3]
    # Else work down the decision tree recurrsively
    else:
        return classify_example(example, response)


In [12]:
def calculate_accuracy(df, tree):
    """
    calculate_accuracy:
    Takes the dataframe sent and classifies each row. Then it compares the classification
    to the actual label and calculates the accuacy. 
    """
    df['classification'] = df.apply(classify_example, axis= 1, args = (tree,))
    df['class_true'] = df.classification == df.label
    accuracy = df['class_true'].sum()/len(df)
    return accuracy

In [13]:
def print_decision_tree(tree, tabs = 0):
    """
    print_decision_tree:
    Takes the decision tree and makes it readable for the user with tabs based on what tree we are on. 
    It will display the decision and leaf node. 
    """
    # tab is used to determine the number of tabs to insert based on what part of the sub tree we are on
    for decision,leafs in tree.items():
        print('{}{}'.format('\t' * tabs, decision))
        tabs += 1    
        for leaf in leafs:
            if type(leaf) != dict:
                print('{}{}'.format('\t' * tabs,leaf))
            else:
                print_decision_tree(leaf, tabs)

In [14]:
# get training and test dataframes
train_df, test_df = train_test_split(data, .2)
tree = decision_tree_algorithm(train_df)
print('The decision tree was {}% accurate.\n'.format(calculate_accuracy(test_df,tree)*100))
print_decision_tree(tree)

The decision tree was 96.66666666666667% accurate.

Decision: petal_width <= 0.8 , Gain = 0.941291028227705
	Leaf: label = Iris-setosa purity = 1.0 size = 43
	Decision: petal_width <= 1.75 , Gain = 0.7140999395534506
		Decision: petal_length <= 4.95 , Gain = 0.14962749466371295
			Leaf: label = Iris-versicolor purity = 0.972972972972973 size = 37
			Leaf: label = Iris-virginica purity = 0.6666666666666666 size = 3
		Leaf: label = Iris-virginica purity = 0.972972972972973 size = 37
